In [3]:
%%classpath add mvn 
org.nd4j nd4j-native-platform 0.7.2
org.nd4j nd4s_2.11 0.7.2

In [2]:
%classpath add jar ../target/scala-2.11/scala-miniflow_2.11-0.1.0-SNAPSHOT.jar

In [3]:
import com.github.timsetsfire.nn.node._
import com.github.timsetsfire.nn.activation._
import com.github.timsetsfire.nn.costfunctions._
import com.github.timsetsfire.nn.regularization._
import com.github.timsetsfire.nn.optimize._
import com.github.timsetsfire.nn.graph._

import scala.util.Try
import org.nd4j.linalg.factory.Nd4j
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.ops.transforms.Transforms.{sigmoid,exp,log}
import org.nd4s.Implicits._
import scala.collection.mutable.{ArrayBuffer, Map=>MutMap, Set=>MutSet}
OutputCell.HIDDEN

In [4]:
val d = Nd4j.readNumpy("../data/mnist_test.csv", ",")
val Array(nrows, ncols) = d.shape
val x_ = d(->, 1 until ncols)
val y_ = d(->, 0)
OutputCell.HIDDEN

In [5]:
val ypos = y_.dup.data.asInt zipWithIndex
val y2_ = Nd4j.zeros(nrows, 10)
ypos.foreach{ case (k,v) => y2_.putScalar(v,k,1.0)}
OutputCell.HIDDEN

Create Network

In [16]:

val images = new Input()
images.setName("images")
val labels = new Input()
labels.setName("labels")

// discriminator

val h1Discrim = LeakyReLU(images, (784,256), 0.1)
h1Discrim.setName("discriminator_hidden_layer1")
val d1 = new Dropout(h1Discrim, 0.01)
d1.setName("dropout_h1_layer")

val h2Discrim = LeakyReLU(d1, (256,64), 0.1)
h2Discrim.setName("discriminator_hidden_layer2")
val d2 = new Dropout(h2Discrim,0.01)
d2.setName("dropout_h2_layer")

val h3Discrim = LeakyReLU(d2, (64,16), 0.1)
h3Discrim.setName("discriminator_hidden_layer3")

val logits = Linear(h3Discrim, (16, 10))
logits.setName("discriminator_logits")

val ce = new CrossEntropyWithLogits(labels, logits)

OutputCell.HIDDEN


In [17]:
    def setDropoutTraining(n: Node, training: Boolean = false): Unit = {
      n.asInstanceOf[Dropout[Node]].train = training
    }

setDropoutTraining: (n: com.github.timsetsfire.nn.node.Node, training: Boolean)Unit


In [18]:

val network = topologicalSort{ 
    buildGraph(ce)
}



[[Variable@69c556f, Variable@69b764af, Variable@1680b243, {labels}@4b9024ea, Variable@72d3b65b, Variable@7c9a741c, Variable@28e20014, {images}@4e472edd, Variable@139464ba, Variable@44402e20, Linear@2eb000b, {discriminator_hidden_layer1}@42d2026f, {dropout_h1_layer}@4c986f35, Linear@3effed60, {discriminator_hidden_layer2}@3bde91bd, {dropout_h2_layer}@71a35558, Linear@11efd576, {discriminator_hidden_layer3}@2330f079, {discriminator_logits}@2c5ec2ed, CrossEntropyWithLogits@54622b2e]]

In [19]:

val epochs = 20
val batchSize = 600
val stepsPerEpoch = nrows / batchSize


network.foreach{
  node =>
  if(node.getClass.getSimpleName == "Variable" ) {
        val (m,n) = node.size
        node.value = Nd4j.randn(m.asInstanceOf[Int], 
                                n.asInstanceOf[Int]
                               ).div(math.sqrt(m.asInstanceOf[Int]) + math.sqrt(n.asInstanceOf[Int] ))
  }
}


network.filter( _.getClass.getSimpleName == "Dropout").map{ i => setDropoutTraining(i, true) }



[[(), ()]]

In [20]:
val data = Nd4j.concat(1,y2_, x_)
OutputCell.HIDDEN

In [21]:
data.shape

[10000, 794]

In [22]:
data.getRow(0).getColumns( (10 until (784+10)):_*).shape

[1, 784]

In [23]:
def accuracy(y: INDArray, logits: INDArray) = {
    val p = exp(logits)
    p.diviColumnVector(p.sum(1))
    val yhat_ = Nd4j.argMax(p,1)
    (Nd4j.argMax(y, 1) eq yhat_).sum(0) / y_.shape.apply(0).toDouble
}

accuracy: (y: org.nd4j.linalg.api.ndarray.INDArray, logits: org.nd4j.linalg.api.ndarray.INDArray)org.nd4j.linalg.api.ndarray.INDArray


In [24]:

val adam = new Adam(network)

var cost = 0d
var n = 0d
for(i <- 0 to epochs) {

    var loss = 0d
    for(j <- 0 until stepsPerEpoch) {

        Nd4j.shuffle(data, 1)

        val feedDict: Map[Node, Any] = Map(
            images -> data.getColumns( (10 until 10 + 784):_*).getRows( (0 until batchSize):_*),
            labels -> data.getColumns( (0 until 10):_*).getRows( (0 until batchSize):_*)
        )

        adam.optimize(feedDict)
        val n1 = images.value.shape.apply(0)
        val c1 = ce.value.sumT * n1
        cost += c1
        n += n1
        
    }
    if(i % 10 == 0) println(f"epoch: ${i}, loss: ${cost / n.toDouble}, accuracy: ${accuracy(labels.value, logits.value)}")
}



epoch: 0, loss: 1.7254611141979694, accuracy: 0.05
epoch: 10, loss: 0.3509552170607177, accuracy: 0.06
epoch: 20, loss: 0.20189838504995264, accuracy: 0.06


null

In [25]:
network.filter( _.getClass.getSimpleName == "Dropout").map{ i => setDropoutTraining(i, false) }

val feedDict: Map[Node, INDArray] = Map( 
    images -> data.getColumns( (10 until 10 + 784):_*),
    labels -> data.getColumns( (0 until 10):_*)
)
feedDict.foreach{ case (node, value) => node.forward(value)}
network.foreach( _.forward())

println(f"accuracy ${accuracy(labels.value, logits.value)}")

accuracy 1.00


null

In [ ]:
import java.io._

@SerialVersionUID(100L)
class SNode(val inboundNodes: SNode*) extends Serializable 
class TNode(inboundNodes: TNode*) extends SNode(inboundNodes:_*)

In [ ]:


    // (1) create a Stock instance
    val x = new TNode()
    val y = new TNode(x)


    //val net = topologicalSort( buildGraph(y) )

//net.getClass
    



In [ ]:
val t = ArrayBuffer(x,y)

In [ ]:
val y = ArrayBuffer(1,2,3,4)
    // (2) write the instance out to a file
    val oos = new ObjectOutputStream(new FileOutputStream("test.srl"))
    oos.writeObject(t)
    oos.close

    // (3) read the object back in
    val ois = new ObjectInputStream(new FileInputStream("test.srl"))
    val xin = ois.readObject.asInstanceOf[ArrayBuffer[TNode]]
    ois.close

    // (4) print the object that was read back in
    //println(xin.value)



In [ ]:
x.getClass.getName

In [ ]:
val test= ArrayBuffer(1,2,3)

In [ ]:
test -- 1

In [ ]:

test - 1

In [ ]:
test.foreach{ i => test - i}

In [ ]:
test